In [1]:
import string
import pandas as pd
import re
from nltk.corpus import stopwords

In [2]:
# Change this variable and the path in the else statement below.
laurence = True
path = None
if laurence:
    path = "C:/Users/laure/OneDrive/Documenten/Project 3.1 APG Files/"
else:
    path = "C:/Users/01din/OneDrive/Documenten/apg-data/"

private_scrape = pd.read_excel(path + "private_scrape_results.xlsx")
public_scrape = pd.read_excel(path + "public_scrape_results.xlsx")

In [3]:
# Storing all of the records of 'public_scrape' in English in a separate dataframe. 'private_scrape' does not contain any records in
# English so this is not needed there.
public_scrape_english = public_scrape.loc[public_scrape['full_url'].str.contains("english", case=False)]

# Storing all of the records of 'public_scrape' in Dutch in a separate dataframe.
public_scrape_dutch = public_scrape.loc[~public_scrape['full_url'].str.contains("english", case=False)]

# Obtaining the paragraphs of these scrape results and converting them to lists, making them easier to iterare over.
private_corpus = private_scrape.paragraph.tolist()
public_corpus = public_scrape_dutch.paragraph.tolist()
public_corpus_english = public_scrape_english.paragraph.tolist()

# Adding the 'private_corpus' and 'public_corpus' as they are both in Dutch.
corpus_dutch = public_corpus + private_corpus
corpus_english = public_corpus_english

# Defining which words in Dutch we want to remove before starting computing the tf-idf matrix.
dutch_stopwords = stopwords.words('dutch')
dutch_months = ['Januari', "Februari", "Maart", "April", "Mei", "Juni", "Juli", "Augustus", "September", "Oktober", "November", "December", 'januari', "februari", "maart", "april", "mei", "juni", "juli", "augustus", "september", "oktober", "november", "december"]
words_to_remove_dutch = dutch_stopwords + dutch_months

# Defining which words in English we want to remove before starting computing the tf-idf matrix.
english_stopwords = stopwords.words('english')
english_months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December", "january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
words_to_remove_english = english_stopwords + english_months

In [32]:
def preprocess_corpus(corpus, words_to_remove, dutch):
    all_words = []
    sentences = []

    for doc in corpus:
        sentences.extend(re.split('[.|?|!|;|\n|\t]', str(doc)))

    for index, sentence in enumerate(corpus):
        sentence = str(sentence).lower()
        sentence = sentence.replace("á", "a").replace("é", "e").replace("è", "e").replace("ó", "o").replace("ò", "o").replace("í", "i")
        sentence = sentence.translate(str.maketrans("", "", (string.punctuation + "’")))
        sentence = "".join([i for i in sentence if not i.isdigit()])
        while "  " in sentence:
            sentence = sentence.replace("  ", " ")
        all_words += (sentence.split())
    all_words = list(set(all_words))

    
    for index, sentence in enumerate(corpus):
        sentence = str(sentence).lower()
        sentence = sentence.replace("á", "a").replace("é", "e").replace("è", "e").replace("ó", "o").replace("ò", "o").replace("í", "i")
        sentence = sentence.translate(str.maketrans("", "", (string.punctuation + "’")))
        sentence = "".join([i for i in sentence if not i.isdigit()])
        while "  " in sentence:
            sentence = sentence.replace("  ", " ")
        
        words = sentence.split()
        stemmed_sentence = []
        
        for word in words:
            if "xd" in word and word != 'exdeelnemers':
                word = word.replace("xd", "")
            if word not in words_to_remove:
                if dutch:
                    if word.endswith("ing") and word[:-3] in all_words:
                        stemmed_sentence.append(word[:-3])
                    elif word.endswith("s") and word[:-1] in all_words:
                        stemmed_sentence.append(word[:-1])
                    elif word.endswith("ig") and word[:-2] in all_words:
                        stemmed_sentence.append(word[:-2])
                    elif word.endswith("isme") and word[:-4] in all_words:
                        stemmed_sentence.append(word[:-4])
                    elif word.endswith("lijk") and word[:-4] in all_words:
                        stemmed_sentence.append(word[:-4])
                    elif word.endswith("e") and word[:-1] in all_words:
                        stemmed_sentence.append(word[:-1])
                    elif word.endswith("en") and word[:-2] in all_words:
                        stemmed_sentence.append(word[:-2])
                    else:
                        stemmed_sentence.append(word)
                elif not dutch:
                    if word.endswith("s") and word[:-1] in all_words:
                        stemmed_sentence.append(word[:-1])
                    elif word.endswith("ism") and word[:-3] in all_words:
                        stemmed_sentence.append(word[:-3])
                    elif word.endswith("ed") and word[:-2] in all_words:
                        stemmed_sentence.append(word[:-2])
                    elif word.endswith("al") and word[:-2] in all_words:
                        stemmed_sentence.append(word[:-2])
                    elif word.endswith("ist") and word[:-3] in all_words:
                        stemmed_sentence.append(word[:-3])
                    elif word.endswith("ity") and word[:-3] in all_words:
                        stemmed_sentence.append(word[:-3])
                    elif word.endswith("ness") and word[:-4] in all_words:
                        stemmed_sentence.append(word[:-4])
                    else:
                        stemmed_sentence.append(word)

        stemmed_sentence = " ".join(stemmed_sentence)
        sentences[index] = stemmed_sentence
    return sentences, all_words



In [33]:
dutch_sentences, all_words_dutch = preprocess_corpus(corpus_dutch, words_to_remove_dutch, True)
english_sentences, all_words_english = preprocess_corpus(corpus_english, words_to_remove_english, False)

IndexError: list assignment index out of range

In [16]:
dff = pd.DataFrame(all_words_dutch, columns=['all_words'])
dff.to_csv(path + 'all_words_dutch.csv', index=False)

dfe = pd.DataFrame(all_words_english, columns=['all_words'])
dfe.to_csv(path + 'all_words_english.csv', index=False)

In [20]:
dutch_sentences.extend(english_sentences)
all_sentences = dutch_sentences

In [21]:
# Convert to dataframe
processed_df = pd.DataFrame(all_sentences, columns=['sentences'])

In [ ]:
# Save to csv
filename = 'public_and_private_sentences_filtered.csv'
if laurence:
    processed_df.to_csv(path + filename, index=False)
else:
    processed_df.to_csv('DATA/scrape_results/' + filename, index=False)


In [22]:
keywords_tf_idf = pd.read_csv(path + "tf_idf_keywords_try.csv")

In [23]:
keywords_tf_idf = keywords_tf_idf["0"].tolist()

In [24]:
counter = 0
list_words = []
for sentence in all_sentences:
    words = sentence.split()
    for word in words:
        list_words.append(word)

for keyword in keywords_tf_idf:
    if keyword not in list_words:
        counter += 1
        print(keyword)



print(counter)

aan
aanbevelingen
aansluiten
aarde
abpcommunicatieplannernl
abpnl
abpnlanwcompensatie
abpnlarbeidsongeschiktheid
abpnldereisvanjeleven
abpnldigid
abpnlkeuzehulp
abpnlmail
abpnlmeerwaarde
abpnlnabestaanden
abpnlpensioen
abpnlpensioninformation
abpnlverkiezingen
abpnlverlagen
abpomdat
abppensioenspelnl
activeertlees
adajavloetabpeu
ai
al
als
altijd
andere
anderen
anders
appvia
armen
ashakhoenkhoenabpeu
aub
bedragen
bedreigingen
begrijpen
behaalde
belangrijkekeuzesnl
belastingdienstnl
beleggingen
beleggingstips
bemoei
bereiken
bertram
beste
bevoegde
bloedingen
boeken
bolcom
bordjes
bouwen
brdugmabpnl
brengen
bufferverplichting
businessasusual
catastrophe
challengebased
clubs
clueless
collegas
daar
dan
dat
dbetonprinten
deelnemers
demping
den
denken
diensten
digidnl
digidvraag
digitale
dingen
diplomas
discretionaire
dit
diverse
dobzabpnl
docenten
doen
dominiquedijkhuisabpeu
donkergroene
doodgaan
door
dus
economische
ed
een
eerlijke
eigenlijk
eindelijk
enienl
erkende
ervaringen
esgroadmap
e

In [31]:
print(keywords_tf_idf[17074])


όόk


In [34]:
keywords_tf_idf_new = pd.read_csv(path + "tf_idf_keywords_try.csv")
keywords_tf_idf_new = keywords_tf_idf_new["0"].tolist()

keywords_tf_idf = pd.read_csv(path + "tf_idf_keywords.csv")
keywords_tf_idf = keywords_tf_idf["0"].tolist()


In [36]:
difference = [x for x in keywords_tf_idf_new if x not in keywords_tf_idf]
print(difference)

[]
